In [0]:
'''!pip install tensorflow==2.0
!pip install tensorflow_hub
!pip install bert-for-tf2
!pip install sentencepiece'''

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

In [0]:
import bert
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
from tensorflow import keras
import math
import re
import numpy as np

In [0]:
checkpoint_path = '/content/model.ckpt'
max_seq_length = 128  # Your choice here.
bertOutDim = 768

In [0]:
label_dict = {'-NONE':1, 'PDT': 2, 'PAP': 3, 'POP': 4, 'A-C': 5, 'CDC': 6, 'IMSAI': 7, 'NP-PRD': 8, 'ADVP-TMP': 9, 'NP-LOC': 10, 'NESB': 11, 'LRB': 12, 'NP-VOC': 13, 'NBC': 14, 'III': 15, 'OK': 16, 'SBAR-LOC-PRD': 17, 'RATE': 18, 'EST': 19, 'MD': 20, 'GHS': 21, 'LANDOR': 22, 'RP': 23, 'C': 24, 'PP-PUT': 25, 'ADVP-LOC-CLR': 26, 'JJS': 27, 'FTC': 28, 'BMP': 29, 'SYM': 30, 'COPPER': 31, 'G': 32, 'LONDON': 33, 'N': 34, 'WTVJ': 35, 'EC': 36, 'ADJP': 37, 'COMMUNICATIONS': 38, 'CS': 39, 'NCR': 40, 'FALL': 41, 'PLO': 42, 'CAT': 43, 'DDB': 44, 'E': 45, 'IBC': 46, 'CIA': 47, 'V': 48, 'FT': 49, 'PLC': 50, 'PRP': 51, 'NX-TTL': 52, 'NNPS': 53, 'BIRDS': 54, 'DT': 55, 'VBZ': 56, 'SEC': 57, 'FUNDS': 58, 'RATES': 59, 'AND': 60, 'ECONOMIC': 61, 'LTV': 62, 'NP-TPC': 63, 'ROGERS': 64, 'FAMILY': 65, 'PP-LOC-CLR-TPC': 66, 'A-D': 67, 'RNR': 68, 'SBARQ-NOM': 69, 'ADR': 70, 'NP-LGS': 71, 'SALARIES': 72, 'ASSOCIATES': 73, 'MONEY': 74, 'LST': 75, 'ADVP-EXT': 76, 'S-SBJ': 77, 'B': 78, 'ADVP-PRD': 79, 'RRC': 80, 'ADVP-PRP': 81, 'WHADJP': 82, 'JUDICIAL': 83, 'PRODUCTS': 84, 'DIALING': 85, 'PP-PRD-LOC': 86, 'OVER': 87, 'ADVP-PRD-LOC': 88, 'FRAG-TPC': 89, 'SQ-TPC': 90, 'SBAR-NOM-SBJ': 91, 'AGREES': 92, 'ADVP-TMP-CLR': 93, 'F': 94, 'OTC': 95, 'FRAG': 96, 'TRIAL': 97, 'LYNCH': 98, 'ADVP-CLR': 99, 'HUD': 100, 'PP-TTL': 101, 'ADJP-TMP': 102, 'LS': 103, 'DES': 104, 'USA': 105, 'US': 106, 'UCP-PRD': 107, 'S-CLF-TPC': 108, 'ASSETS': 109, 'NL': 110, 'JUDGE': 111, 'SBARQ-TPC': 112, 'SBAR': 113, 'S-PRP-PRD': 114, 'NASD': 115, 'NP-TMP': 116, 'MORTGAGE': 117, 'ABA': 118, 'CONJP': 119, 'CHANGED': 120, 'VBD': 121, 'AT': 122, 'ADVP': 123, 'TRANSPLANT': 124, 'PP-CLR': 125, 'PP-BNF': 126, 'VB': 127, 'GRAINS': 128, 'POTABLES': 129, 'UAL': 130, 'WRB': 131, 'THREE': 132, 'ABORTION': 133, 'NOT': 134, 'PP-TMP-PRD': 135, 'WHPP': 136, 'IBM': 137, 'S-TTL': 138, 'IRA': 139, 'NP-HLN': 140, 'FRAG-TTL-SBJ': 141, 'NEC': 142, 'CBS': 143, 'NBI': 144, 'ADVP-MNR': 145, 'A': 146, 'DOT': 147, 'SINV-TPC': 148, 'UBS': 149, 'SALT': 150, 'PP-DIR': 151, 'CD': 152, 'PPA': 153, 'S-TMP': 154, 'ENERGY': 155, 'SERVICES': 156, 'SAT': 157, 'ADJP-PRD': 158, 'TEXAS': 159, 'TWO': 160, 'MEDICINE': 161, 'SBAR-CLR': 162, 'FAX': 163, 'PORTING': 164, 'PS': 165, 'SBAR-MNR': 166, 'FRAG-HLN': 167, 'UCP': 168, 'SBAR-TMP': 169, 'USIA': 170, 'BALLOT': 171, 'MERRILL': 172, 'GROWTH': 173, 'S-MNR': 174, 'OSHA': 175, 'WP': 176, 'FW': 177, 'BANKERS': 178, 'PP-PRP': 179, 'ADVP-LOC': 180, 'TRS': 181, 'PP-CLR-LOC': 182, 'EXP': 183, 'INS': 184, 'S-PRP-CLR': 185, 'INTER-TEL': 186, 'SWITCHING': 187, 'UNION': 188, 'PAPER': 189, 'INTJ': 190, 'ORTEGA': 191, 'LOAN': 192, 'DD': 193, 'AG': 194, 'UH': 195, 'RULING': 196, 'METALS': 197, 'PP-EXT': 198, 'ADJP-PRD-TPC': 199, 'NP-CLR': 200, 'WAR': 201, 'II': 202, 'SINV': 203, 'VBG': 204, 'SBAR-LOC': 205, 'CEO': 206, 'CLEARS': 207, 'IRS': 208, 'GOODY': 209, 'NAC': 210, 'USX': 211, 'TRIMMING': 212, 'TREASURY': 213, 'CSV': 214, 'X': 215, 'AN': 216, 'LSI': 217, 'DISCOUNT': 218, 'SBAR-PRD': 219, 'WHNP': 220, 'IX': 221, 'COMPUTERS': 222, 'PP-LOC-CLR': 223, 'GM': 224, 'NATIONAL': 225, 'ADVP-DIR': 226, 'YWCA': 227, 'CC': 228, 'NP-MNR': 229, 'OF': 230, 'VP-TPC': 231, 'EVERYONE': 232, 'H': 233, 'S-NOM-PRD': 234, 'TIRED': 235, 'ADVP-PRD-TPC': 236, 'X-HLN': 237, 'COLLECTING': 238, 'WPP': 239, 'S-HLN': 240, 'INQUIRY': 241, 'RCB': 242, 'NNP': 243, 'SBAR-NOM-PRD': 244, 'PP-LOC-TPC': 245, 'WAFA': 246, 'RB': 247, 'POS': 248, 'FIRST': 249, 'K': 250, 'NNS': 251, 'NP-TMP-HLN': 252, 'JJR': 253, 'LIBOR': 254, 'FERC': 255, 'TO': 256, 'WHADVP': 257, 'ASSOCIATION': 258, 'WTD': 259, 'DSM': 260, 'NAC-LOC': 261, 'INTERBANK': 262, 'SBAR-ADV': 263, 'PP-TPC': 264, 'LCB': 265, 'TV': 266, 'S-CLF': 267, 'CALL': 268, 'IOU': 269, 'PP-TMP': 270, 'ADJP-CLR': 271, 'Q': 272, 'FRAG-ADV': 273, 'S-NOM': 274, 'RBS': 275, 'SDI': 276, 'EURODOLLARS': 277, 'SHAREDATA': 278, 'SOYBEANS': 279, 'LATE': 280, 'NEW': 281, 'S-TPC': 282, 'NONE': 283, 'ACQUISITION': 284, 'RRB': 285, 'ISSUES': 286, 'PP-MNR': 287, 'ON': 288, 'BTR': 289, 'DIAPER': 290, 'S-CLR': 291, 'NP-EXT': 292, 'CAMPAIGN': 293, 'ASLACTON': 294, 'PP-LGS': 295, 'PP-TMP-CLR': 296, 'NTG': 297, 'NP-TMP-CLR': 298, 'PP-PRD': 299, 'YMCA': 300, 'APPEARS': 301, 'P': 302, 'READY': 303, 'W': 304, 'PRIME': 305, 'BILLS': 306, 'S-ADV': 307, 'Z': 308, 'SCI': 309, 'U': 310, 'PRT': 311, 'ENDED': 312, 'D': 313, 'TROUBLES': 314, 'ADVP-LOC-PRD': 315, 'INTERPUBLIC': 316, 'PP-DTV': 317, 'FOREIGN': 318, 'PHOTOGRAPH': 319, 'NP-SBJ': 320, 'ACCOUNT': 321, 'UPHELD': 322, 'PP-LOC-PRD': 323, 'FBI': 324, 'NAC-TMP': 325, 'ADVP-LOC-PRD-TPC': 326, 'ADVP-DIR-TPC': 327, 'MITI': 328, 'S-PRP': 329, 'CORP': 330, 'SBARQ': 331, 'NCNB': 332, 'NP': 333, 'GMAC': 334, 'ACCEPTANCES': 335, 'PP': 336, 'NP-TTL-PRD': 337, 'IT': 338, 'SBAR-PRP': 339, 'PP-DIR-CLR': 340, 'ADJP-ADV': 341, 'VP': 342, 'DOONESBURY': 343, 'VBN': 344, 'NP-TMP-TPC': 345, 'JJ': 346, 'OFFERED': 347, 'ADVP-HLN': 348, 'EX': 349, 'M': 350, 'HOME': 351, 'FEDERAL': 352, 'PRECIOUS': 353, 'UCP-MNR': 354, 'BRAMALEA': 355, 'NP-TTL': 356, 'NN': 357, 'CTB': 358, 'COMMERCIAL': 359, 'GHKM': 360, 'FT-SE': 361, 'NP-PRD-TTL': 362, 'PRN': 363, 'NIH': 364, 'VOA': 365, 'BRIEFS': 366, 'THE': 367, 'S-PRD': 368, 'EXCHANGE': 369, 'NP-ADV': 370, 'PC': 371, 'ICH': 372, 'DEFENSE': 373, 'J': 374, 'SQ': 375, 'SBAR-PRP-PRD': 376, 'O': 377, 'CERTIFICATES': 378, 'CREATOR': 379, 'ADJP-TPC': 380, 'PP-LOC-PRD-TPC': 381, 'SBAR-SBJ': 382, 'ADVP-PUT': 383, 'NYSE': 384, 'NRDC': 385, 'WFRR': 386, 'AC': 387, 'PP-LOC': 388, 'THAT': 389, 'TXO': 390, 'GAF': 391, 'WHAS': 392, 'HHS': 393, 'QP': 394, 'RBR': 395, 'S-NOM-SBJ': 396, 'UCP-PRP': 397, 'DNA': 398, 'AMR': 399, 'UCP-ADV': 400, 'UCP-TMP': 401, 'PETS': 402, 'WDT': 403, 'ADVP-DIR-CLR': 404, 'DEPOSIT': 405, 'T': 406, 'PP-MNR-CLR': 407, 'VBP': 408, 'EEOC': 409, 'L': 410, 'AIDS': 411, 'R': 412, 'IN': 413, 'PTA': 414, 'GOP': 415, 'UCP-LOC-CLR': 416, 'TRUST': 417, 'INGERSOLL-RAND': 418, 'PAPERS': 419, 'RMS': 420, 'NX': 421, 'RBC': 422, 'I': 423, 'Y': 424, 'ITC': 425, 'S': 426, 'NP-TTL-SBJ': 427, 'CTBS': 428, 'SBAR-NOM': 429,'.':430, '!':431, '?':432, '@':433,':':434,'&':435,'+':436,'{':437,'}':438,'[':439,']':440,'$':441,'/':442,'"':443,'-':444,',':445,'``':446,"''":447,'#':448,'ADVP|PRT':449}

In [0]:
totalTags = len(label_dict)

In [0]:
def getBERTModel():
    input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="input_mask")
    segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="segment_ids")

    bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=False)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

    # Import tokenizer using the original vocab file
    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
    FullTokenizer = bert.bert_tokenization.FullTokenizer
    tokenizer = FullTokenizer(vocab_file, do_lower_case)

    return {'model':model,'tokenizer':tokenizer}

In [0]:
def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [0]:
def getEmbeddings(model,tokenizer,sentence): 
    stokens = tokenizer.tokenize(sentence)
    stokens = ["[CLS]"] + stokens + ["[SEP]"]

    input_ids = get_ids(stokens, tokenizer, max_seq_length)
    input_masks = get_masks(stokens, max_seq_length)
    input_segments = get_segments(stokens, max_seq_length)

    pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])
    # pool_ebmbs is an embeding of CLS token
    # all_embs contains embeding for words of input sentence.
    return all_embs[0]

In [0]:
def genSpanEncode(embs,i,j):
    vi = embs[i]
    vj = embs[j]

    fi = vi[::2]
    bi = vi[1::2]
    fj = vj[::2]
    bj = vj[1::2]

    spanEncode = np.concatenate(((fi - fj), (bi - bj)), axis=0)
    #print('span shape',spanEncode.shape)
    return spanEncode

In [0]:
def createModel():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(512,input_dim=bertOutDim,activation="relu"))
    model.add(keras.layers.Dense(totalTags,activation="softmax"))
    model.compile(loss="categorical_crossentropy",optimizer="adam")
    return model

In [0]:
def idTolabel(id=None, label_dict=label_dict):
    id_dict = dict(map(reversed, label_dict.items()))
    if id is not None:
        return id_dict[id]
    else:
        return None

In [0]:
def getMaxLabel(model,embs,i,j):
    spanEnc = genSpanEncode(embs,i,j)
    #print('shape 2',spanEnc.shape)
    out=model.predict(np.asarray([spanEnc]))[0]
    #print(out)
    #print(out.shape)
    '''#list
    score = max(out)
    ind = out.index(score)
    ind = ind + 1'''

    #numpy array
    score = np.amax(out)
    ind = np.where(out == score)[0][0]
    ind = ind + 1 #Because dictionary id start from 1.

    label = idTolabel(ind,label_dict)
    # score = 0 for unkown tag
    '''if ind == 0:
        score = 0'''
    return score,label

In [0]:
def decoder(sentence):
    n = len(sentence.split())

    outModel = getBERTModel()
    bertModel = outModel['model']
    tokenizer = outModel['tokenizer']

    model = createModel()
    print("Model created.")
    model.load_weights(checkpoint_path)

    embs = getEmbeddings(bertModel,tokenizer,sentence)
    print("Embedding generated...")
    M = [[-1 for i in range(n)] for i in range(n)]
    S = [[-1 for i in range(n)] for i in range(n)]
    T = [[0 for i in range(n)] for i in range(n)]

    for i in range(n):
        score,tag = getMaxLabel(model,embs,i,i)
        #print(i,score)
        M[i][i] = score
        T[i][i] = tag

    for l in range(1,n):
        for i in range(n-l):
            j = i + l
            score,tag = getMaxLabel(model,embs,i,j)
            T[i][j] = tag
            for k in range(i,j):
                  q = M[i][k] + M[k+1][j] + score
                  if q > M[i][j]:
                    M[i][j] = q
                    S[i][j] = k
            
    return S,T 

In [0]:
def printTree(S,T,sentence,i,j):
    if i == j:
      print('(', end="")
      print( T[i][j], end=" " )
      print(sentence.split()[i], end="")
      print(')',end="")
    else:
      print('(', end="")
      print( T[i][j] )
      printTree(S,T,sentence,i,S[i][j])
      printTree(S,T,sentence,S[i][j]+1,j)
      print(')',end="")

In [0]:
sentence = "short cuts make long delays"
#sentence = "a lorillard spokewoman said , `` this is an old story ."
n = len(sentence.split())

In [0]:
S,T = decoder(sentence)

In [0]:
#print(S)

In [0]:
print(T)

In [0]:
'''S= [[-1,0,0,0,1],[-1,-1,0,0,0],[-1,-1,-1,0,2],[-1,-1,-1,-1,3],[-1,-1,-1,-1,-1]]
T = [['JJ','NP','NONE','NONE','S'],[0,'NNS','NONE','NONE','NONE'],[0,0,'VBP','NONE','VP'],[0,0,0,'JJ','NP'],[0,0,0,0,'NNS']]'''

In [0]:
printTree(S,T,sentence,0,n-1,)